In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

TEST_MODE = 0 # Testing macro

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

if TEST_MODE:
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install --upgrade pip

import numpy as np # linear algebraf
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import warnings
from sklearn.metrics import roc_curve, make_scorer, auc
from scipy.optimize import brentq
from scipy.interpolate import interp1d



pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns


from pandas.testing import assert_frame_equal


%run ./SEED-CONSTANTS.ipynb

# Global utitlity functions are in separate notebook
print("setup complete")

"SEED and CONSTANTS imported"
setup complete


In [4]:
err_distro_rc["figure.figsize"]

(20, 10)

In [5]:
!python --version

Python 3.9.10


In [6]:
def rolling_window(a, window, stride):
    shape = a.shape[:-1] + (int((a.shape[-1] - window)/stride + 1), window)
    strides = (stride*a.strides[-1],) + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)



def getIndices(sampleSize=1000, step=1000, numSamplePoints=24000):
    indices = np.arange(0, numSamplePoints, 1)
    indices = rolling_window(indices, sampleSize, step)
    
    return indices

In [7]:
# cite the thesis paper i found
def utils_ppp(P):
    """Pretty print parameters of an experiment."""
    df = pd.DataFrame([asdict(P)])
    df = df.T
    df.columns = ["Value"]
    
    display(df)

In [8]:
# source: https://github.com/dynobo/ContinAuth/blob/master/notebooks/utils.ipynb
def utils_eer(y_true, y_pred, return_threshold=False):
    """Calculate the Equal Error Rate.

    Based on https://stackoverflow.com/a/49555212, https://yangcha.github.io/EER-ROC/
    and https://scikit-learn.org/stable/modules/model_evaluation.html#implementing-your-own-scoring-object

    Arguments:
        y_true {np.array}  -- Actual labels
        y_pred {np.array}  -- Predicted labels or probability
        
    Returns:
        float              -- Equal Error Rate        
    """
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)
    eer = brentq(lambda x: 1.0 - x - interp1d(fpr, tpr)(x), 0.0, 1.0)
    thresh = interp1d(fpr, thresholds)(eer)  # Calculated threshold, not needed for score
    if return_threshold:
        return eer, thresh
    else:
        return eer

In [9]:
if TEST_MODE:
    temp_eer, tres = utils_eer(
        [-1, -1, -1, 1, 1, 1], [0, 0.9, 0.1, 0.74, 0.8, .6], return_threshold=True
    )
    print(f"EER: {temp_eer:.3f}, Threshold: {tres:.3f} <-- Arbitrary case")

    temp_eer, tres = utils_eer(
        [-1, -1, -1, 1, 1], [0.1, 0.2, 0.3, 1, 0.9], return_threshold=True
    )
    print(f"EER: {temp_eer:.3f}, Threshold: {tres:.3f} <-- Best case")

    temp_eer, tres = utils_eer(
        [1, 1, 1, -1, -1], [0.1, 0.2, 0.3, 1, 0.9], return_threshold=True
    )
    print(f"EER: {temp_eer:.3f}, Threshold: {tres:.3f} <-- Worse case")
    
#     new case does it make sense? I don't think so
    temp_eer, tres = utils_eer(
        [1, 1, 1, -1, -1], [-1, 1, -1, -1, -1], return_threshold=True
    )
    print(f"EER: {temp_eer:.3f}, Threshold: {tres:.3f} <-- Worse case")

In [10]:
utils_eer_scorer = make_scorer(utils_eer, greater_is_better=False)

In [11]:
# source: https://zhiyzuo.github.io/Plot-Lorenz/
#  0 representing perfect equality, and 1 absolute inequality.
def gini(arr):
    ## first sort
    sorted_arr = arr.copy()
    sorted_arr.sort()
    n = arr.size
    coef_ = 2. / n
    const_ = (n + 1.) / n
    weighted_sum = sum([(i+1)*yi for i, yi in enumerate(sorted_arr)])
    return coef_*weighted_sum/(sorted_arr.sum()) - const_

def lorenz_curve(X):
    ## first sort
    X = X.copy()
    X.sort()
    X_lorenz = X.cumsum() / X.sum()
    X_lorenz = np.insert(X_lorenz, 0, 0) 
    X_lorenz[0], X_lorenz[-1]
    fig, ax = plt.subplots(figsize=[6,6])
    ## scatter plot of Lorenz curve
    ax.scatter(np.arange(X_lorenz.size)/(X_lorenz.size-1), X_lorenz, 
               marker='x', color='darkgreen', s=100)
    ## line plot of equality
    ax.plot([0,1], [0,1], color='k')
    ax.set_xlabel('% of Population')
    ax.set_ylabel('% of Errors')

In [12]:
def getAUROCDist(window_frrList_dict, window_farList_dict, start_window_size=250, end_window_size=3000, increment_step=250):
    
    window_AUROC_dict = {}
    lst = np.arange(start_window_size, end_window_size + 1, increment_step)
    
    for w in lst:
        
        frrList = dic["window_frrList_dict"][w]
        farList = dic["window_farList_dict"][w]
        window_AUROC_dict[w] = auc(frrList, farList)
        
    return window_AUROC_dict

In [13]:
print("\x1b[32m\"Utility functions imported\"\x1b[0m")

"Utility functions imported"
